<a href="https://colab.research.google.com/github/AyeshaIjazTabassum/PythonAIBootcamp/blob/main/Day27.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parallel Chaining

In [1]:
! pip install python-dotenv langchain langchain-core langchain-google-genai

In [2]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableLambda, RunnableParallel
from google.colab import userdata


GOOGLE_API_KEY = userdata.get('API')
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", api_key=GOOGLE_API_KEY)

In [3]:
summary_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie critic."),
        ("human", "Provide a brief summary of the movie {movie_name}."),
    ]
)


In [4]:
# Define plot analysis step
def analyze_plot(plot):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the plot: {plot}. What are its strengths and weaknesses?"),
        ]
    )
    return plot_template.format_prompt(plot=plot)

# Define character analysis step
def analyze_characters(characters):
    character_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a movie critic."),
            ("human", "Analyze the characters: {characters}. What are their strengths and weaknesses?"),
        ]
    )
    return character_template.format_prompt(characters=characters)


In [6]:
# Combine analyses into a final verdict
def combine_verdicts(plot_analysis, character_analysis):
    return f"Plot Analysis:\n{plot_analysis}\n\nCharacter Analysis:\n{character_analysis}"

In [7]:
# Simplify branches with LCEL
plot_branch_chain = (
    RunnableLambda(lambda x: analyze_plot(x)) | llm | StrOutputParser()
)

character_branch_chain = (
    RunnableLambda(lambda x: analyze_characters(x)) | llm | StrOutputParser()
)

In [8]:
chain = (
    summary_template
    | llm
    | StrOutputParser()
    | RunnableParallel(branches={"plot": plot_branch_chain, "characters": character_branch_chain})
    | RunnableLambda(lambda x: combine_verdicts(x["branches"]["plot"], x["branches"]["characters"]))
)


In [9]:
result = chain.invoke({"movie_name": "Inception"})

print(result)

Plot Analysis:
Ah, *Inception*. A film that, much like the dreams it depicts, lingers long after the credits roll, sparking endless debate and fascination. You've laid out a solid foundation of its premise, and now, let's dissect its strengths and weaknesses as a master craftsman would examine a meticulously constructed dreamscape.

## Inception: A Critical Analysis of Its Strengths and Weaknesses

Christopher Nolan's *Inception* is a film that aims for the stars, and for the most part, it reaches them with breathtaking ambition and execution. However, like any complex edifice, it has its structural elements that are undeniably brilliant and others that, upon closer inspection, might reveal a few hairline cracks.

### Strengths: Where the Dream Soars

*   **The Core Concept and Originality:** This is *Inception*'s undeniable trump card. The idea of extracting and planting ideas within dreams is a stroke of genius. It elevates the heist genre into something far more profound and intelle